In [105]:
import os
import sys
import numpy as np
import pandas as pd
import torch
from torch_geometric.data import Data
from torch_geometric.data import Batch
from torch_geometric.data import DataLoader
from torch_geometric.nn import TopKPooling
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch.nn import MSELoss, CrossEntropyLoss
from torch_geometric.nn import GCNConv
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [132]:
global model
model = ""
num_features = 1
num_classes = 6
hc = 16
n_epochs = 20
batch_size = 128

In [107]:
def meanNorm(X):
    dfx = []
    col = X.columns.tolist()
    
    for ix,r in X.iterrows():
        r = pd.Series(r)
        val = r.values.tolist()
        s = sum(val)

        rw = [i/s for i in val]

        dfx.append(rw)
    X = pd.DataFrame(dfx, columns = col)
    return X

def visualize(h, color):
    z = TSNE(n_components=2).fit_transform(out.detach().cpu().numpy())

    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])

    plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
    plt.show()



In [108]:
n0 = pd.read_csv("../data/tracing-data/hotel-reservation/0_frontend.csv")
n0['label'] = 0
n1 = pd.read_csv("../data/tracing-data/hotel-reservation/1_search.csv")
n1['label'] = 1
n2 = pd.read_csv("../data/tracing-data/hotel-reservation/2_geo.csv")
n2['label'] = 2
n3 = pd.read_csv("../data/tracing-data/hotel-reservation/3_rate.csv")
n3['label'] = 3
n4 = pd.read_csv("../data/tracing-data/hotel-reservation/4_profile.csv")
n4['label'] = 4
n5 = pd.read_csv("../data/tracing-data/hotel-reservation/5_locale.csv")
n5['label'] = 5

In [109]:
dat = pd.concat([n0, n1, n2, n3, n4, n5])
print(dat.shape)
dat = dat.iloc[0:10000, :]

y = dat['label']
X = dat[["0_frontend", "1_search", "2_geo", "3_rate", "4_profile", "5_locale"]]

X = meanNorm(X)
print(X.shape)

(4740000, 8)
(10000, 6)


In [110]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_dat = pd.DataFrame(X_train, columns = ["0_frontend", "1_search", "2_geo", "3_rate", "4_profile", "5_locale"])
train_dat['label'] = y_train
test_dat = pd.DataFrame(X_test, columns = ["0_frontend", "1_search", "2_geo", "3_rate", "4_profile", "5_locale"])
test_dat['label'] = y_test


In [113]:
graphs_train = list()
count = 0
for row in train_dat.iterrows():
    r = pd.Series(row)
    r = r[1]
    x = torch.tensor([[r['0_frontend']],
                      [r['1_search']],
                      [r['2_geo']],
                      [r['3_rate']],
                      [r['4_profile']],
                      [r['5_locale']]] ,dtype = torch.float)
    edge_index = torch.tensor([[0, 0, 1, 1, 4],
                               [1, 4, 2, 3, 5]], dtype=torch.long)
    if r['label'] == 0:
        y = torch.tensor([1,0,0,0,0,0], dtype = torch.float)
    elif r['label'] == 1:
        y = torch.tensor([0,1,0,0,0,0], dtype = torch.float)
    elif r['label'] == 2:
        y = torch.tensor([0,0,1,0,0,0], dtype = torch.float)
    elif r['label'] == 3:
        y = torch.tensor([0,0,0,1,0,0], dtype = torch.float)
    elif r['label'] == 4:
        y = torch.tensor([0,0,0,0,1,0], dtype = torch.float)
    elif r['label'] == 5:
        y = torch.tensor([0,0,0,0,0,1], dtype = torch.float)
    else:
        print("unknown label encountered")
        break
    graphs_train.append(Data(x = x, edge_index = edge_index,y = y))
    count = count + 1
    
    
graphs_test = list()
for row in test_dat.iterrows():
    r = pd.Series(row)
    r = r[1]
    x = torch.tensor([[r['0_frontend']],
                      [r['1_search']],
                      [r['2_geo']],
                      [r['3_rate']],
                      [r['4_profile']],
                      [r['5_locale']]] ,dtype = torch.float)
    edge_index = torch.tensor([[0, 0, 1, 1, 4],
                               [1, 4, 2, 3, 5]], dtype=torch.long)
    if r['label'] == 0:
        y = torch.tensor([1,0,0,0,0,0], dtype = torch.float)
    elif r['label'] == 1:
        y = torch.tensor([0,1,0,0,0,0], dtype = torch.float)
    elif r['label'] == 2:
        y = torch.tensor([0,0,1,0,0,0], dtype = torch.float)
    elif r['label'] == 3:
        y = torch.tensor([0,0,0,1,0,0], dtype = torch.float)
    elif r['label'] == 4:
        y = torch.tensor([0,0,0,0,1,0], dtype = torch.float)
    elif r['label'] == 5:
        y = torch.tensor([0,0,0,0,0,1], dtype = torch.float)
    else:
        print("unknown label encountered")
        break
    graphs_test.append(Data(x = x, edge_index = edge_index,y = y))
    count = count + 1


In [133]:
print(f'Number of graphs: {len(graphs_train)}')

data = graphs_train[0]  # Get the first graph object.

print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')


Number of graphs: 8000

Data(edge_index=[2, 5], x=[6, 1], y=[6])
Number of nodes: 6
Number of edges: 5
Average node degree: 0.83


In [134]:
# Initialize scaling and dataloader

scaler = StandardScaler()
loader = DataLoader(graphs, batch_size = batch_size)

In [135]:
print(graphs[0].num_node_features)
print(graphs[0])

1
Data(edge_index=[2, 5], x=[6, 1], y=[6])


In [136]:

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.1, training=self.training)
        x = self.conv2(x, edge_index)
#         return F.softmax(x)
        return x

In [137]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
for d in range(1):
    data = next(iter(loader))
    data = data.to(device)
    print(data)
print(dataset.num_node_features)
model = GCN(hidden_channels = hc).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
loss_func = CrossEntropyLoss()   

print(len(graphs))
print(len(loader))
print(device)

Batch(batch=[768], edge_index=[2, 640], x=[768, 1], y=[768])
1
24000
188
cpu


In [143]:
out = model(data)
print(out.shape)
print(data.y.long())

torch.Size([384, 6])
tensor([1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 0,

In [142]:
count = 0
for data in loader:
    data = data.to(device)
    count = count + 1
    print(count)
    for epochs in range(n_epochs):
        optimizer.zero_grad()                                                   
        out = model(data) 
        loss = loss_func(out, data.y.long())
        loss.backward()                                                         
        optimizer.step()
        print(f'Epoch: {epochs:03d}, Loss: {loss:.4f}')

1
Epoch: 000, Loss: 1.8550
Epoch: 001, Loss: 1.8021
Epoch: 002, Loss: 1.7570
Epoch: 003, Loss: 1.7141
Epoch: 004, Loss: 1.6690
Epoch: 005, Loss: 1.6274
Epoch: 006, Loss: 1.5871
Epoch: 007, Loss: 1.5392
Epoch: 008, Loss: 1.4924
Epoch: 009, Loss: 1.4485
Epoch: 010, Loss: 1.4036
Epoch: 011, Loss: 1.3639
Epoch: 012, Loss: 1.3225
Epoch: 013, Loss: 1.2820
Epoch: 014, Loss: 1.2420
Epoch: 015, Loss: 1.1917
Epoch: 016, Loss: 1.1579
Epoch: 017, Loss: 1.1134
Epoch: 018, Loss: 1.0815
Epoch: 019, Loss: 1.0373
2
Epoch: 000, Loss: 0.9977
Epoch: 001, Loss: 0.9644
Epoch: 002, Loss: 0.9333
Epoch: 003, Loss: 0.8963
Epoch: 004, Loss: 0.8675
Epoch: 005, Loss: 0.8339
Epoch: 006, Loss: 0.8045
Epoch: 007, Loss: 0.7721
Epoch: 008, Loss: 0.7413
Epoch: 009, Loss: 0.7269
Epoch: 010, Loss: 0.7073
Epoch: 011, Loss: 0.6797
Epoch: 012, Loss: 0.6664
Epoch: 013, Loss: 0.6542
Epoch: 014, Loss: 0.6314
Epoch: 015, Loss: 0.6120
Epoch: 016, Loss: 0.5981
Epoch: 017, Loss: 0.5813
Epoch: 018, Loss: 0.5724
Epoch: 019, Loss: 0.5

Epoch: 008, Loss: 0.2768
Epoch: 009, Loss: 0.2791
Epoch: 010, Loss: 0.2779
Epoch: 011, Loss: 0.2775
Epoch: 012, Loss: 0.2737
Epoch: 013, Loss: 0.2775
Epoch: 014, Loss: 0.2763
Epoch: 015, Loss: 0.2738
Epoch: 016, Loss: 0.2716
Epoch: 017, Loss: 0.2764
Epoch: 018, Loss: 0.2810
Epoch: 019, Loss: 0.2708
18
Epoch: 000, Loss: 0.2571
Epoch: 001, Loss: 0.2537
Epoch: 002, Loss: 0.2573
Epoch: 003, Loss: 0.2537
Epoch: 004, Loss: 0.2556
Epoch: 005, Loss: 0.2581
Epoch: 006, Loss: 0.2510
Epoch: 007, Loss: 0.2528
Epoch: 008, Loss: 0.2499
Epoch: 009, Loss: 0.2503
Epoch: 010, Loss: 0.2509
Epoch: 011, Loss: 0.2487
Epoch: 012, Loss: 0.2505
Epoch: 013, Loss: 0.2464
Epoch: 014, Loss: 0.2478
Epoch: 015, Loss: 0.2533
Epoch: 016, Loss: 0.2527
Epoch: 017, Loss: 0.2436
Epoch: 018, Loss: 0.2515
Epoch: 019, Loss: 0.2475
19
Epoch: 000, Loss: 0.2915
Epoch: 001, Loss: 0.2931
Epoch: 002, Loss: 0.2812
Epoch: 003, Loss: 0.2900
Epoch: 004, Loss: 0.2964
Epoch: 005, Loss: 0.2988
Epoch: 006, Loss: 0.2960
Epoch: 007, Loss: 0

Epoch: 014, Loss: 0.2649
Epoch: 015, Loss: 0.2768
Epoch: 016, Loss: 0.2664
Epoch: 017, Loss: 0.2706
Epoch: 018, Loss: 0.2533
Epoch: 019, Loss: 0.2706
34
Epoch: 000, Loss: 0.2668
Epoch: 001, Loss: 0.2637
Epoch: 002, Loss: 0.2733
Epoch: 003, Loss: 0.2526
Epoch: 004, Loss: 0.2634
Epoch: 005, Loss: 0.2677
Epoch: 006, Loss: 0.2580
Epoch: 007, Loss: 0.2685
Epoch: 008, Loss: 0.2643
Epoch: 009, Loss: 0.2493
Epoch: 010, Loss: 0.2656
Epoch: 011, Loss: 0.2680
Epoch: 012, Loss: 0.2647
Epoch: 013, Loss: 0.2608
Epoch: 014, Loss: 0.2695
Epoch: 015, Loss: 0.2634
Epoch: 016, Loss: 0.2743
Epoch: 017, Loss: 0.2654
Epoch: 018, Loss: 0.2646
Epoch: 019, Loss: 0.2672
35
Epoch: 000, Loss: 0.2494
Epoch: 001, Loss: 0.2484
Epoch: 002, Loss: 0.2434
Epoch: 003, Loss: 0.2444
Epoch: 004, Loss: 0.2492
Epoch: 005, Loss: 0.2470
Epoch: 006, Loss: 0.2458
Epoch: 007, Loss: 0.2417
Epoch: 008, Loss: 0.2468
Epoch: 009, Loss: 0.2430
Epoch: 010, Loss: 0.2467
Epoch: 011, Loss: 0.2452
Epoch: 012, Loss: 0.2485
Epoch: 013, Loss: 0

Epoch: 007, Loss: 0.2344
Epoch: 008, Loss: 0.2358
Epoch: 009, Loss: 0.2371
Epoch: 010, Loss: 0.2461
Epoch: 011, Loss: 0.2433
Epoch: 012, Loss: 0.2399
Epoch: 013, Loss: 0.2329
Epoch: 014, Loss: 0.2313
Epoch: 015, Loss: 0.2376
Epoch: 016, Loss: 0.2359
Epoch: 017, Loss: 0.2309
Epoch: 018, Loss: 0.2297
Epoch: 019, Loss: 0.2226
51
Epoch: 000, Loss: 0.2527
Epoch: 001, Loss: 0.2535
Epoch: 002, Loss: 0.2535
Epoch: 003, Loss: 0.2569
Epoch: 004, Loss: 0.2512
Epoch: 005, Loss: 0.2564
Epoch: 006, Loss: 0.2484
Epoch: 007, Loss: 0.2600
Epoch: 008, Loss: 0.2521
Epoch: 009, Loss: 0.2447
Epoch: 010, Loss: 0.2584
Epoch: 011, Loss: 0.2519
Epoch: 012, Loss: 0.2447
Epoch: 013, Loss: 0.2411
Epoch: 014, Loss: 0.2530
Epoch: 015, Loss: 0.2582
Epoch: 016, Loss: 0.2606
Epoch: 017, Loss: 0.2501
Epoch: 018, Loss: 0.2669
Epoch: 019, Loss: 0.2493
52
Epoch: 000, Loss: 0.2608
Epoch: 001, Loss: 0.2586
Epoch: 002, Loss: 0.2493
Epoch: 003, Loss: 0.2558
Epoch: 004, Loss: 0.2674
Epoch: 005, Loss: 0.2691
Epoch: 006, Loss: 0

Epoch: 015, Loss: 0.2397
Epoch: 016, Loss: 0.2456
Epoch: 017, Loss: 0.2503
Epoch: 018, Loss: 0.2505
Epoch: 019, Loss: 0.2485
67
Epoch: 000, Loss: 0.2414
Epoch: 001, Loss: 0.2501
Epoch: 002, Loss: 0.2388
Epoch: 003, Loss: 0.2500
Epoch: 004, Loss: 0.2456
Epoch: 005, Loss: 0.2351
Epoch: 006, Loss: 0.2351
Epoch: 007, Loss: 0.2230
Epoch: 008, Loss: 0.2421
Epoch: 009, Loss: 0.2340
Epoch: 010, Loss: 0.2432
Epoch: 011, Loss: 0.2437
Epoch: 012, Loss: 0.2316
Epoch: 013, Loss: 0.2526
Epoch: 014, Loss: 0.2369
Epoch: 015, Loss: 0.2403
Epoch: 016, Loss: 0.2388
Epoch: 017, Loss: 0.2333
Epoch: 018, Loss: 0.2284
Epoch: 019, Loss: 0.2380
68
Epoch: 000, Loss: 0.2542
Epoch: 001, Loss: 0.2414
Epoch: 002, Loss: 0.2458
Epoch: 003, Loss: 0.2587
Epoch: 004, Loss: 0.2513
Epoch: 005, Loss: 0.2536
Epoch: 006, Loss: 0.2352
Epoch: 007, Loss: 0.2465
Epoch: 008, Loss: 0.2404
Epoch: 009, Loss: 0.2448
Epoch: 010, Loss: 0.2435
Epoch: 011, Loss: 0.2438
Epoch: 012, Loss: 0.2515
Epoch: 013, Loss: 0.2504
Epoch: 014, Loss: 0

...

Epoch: 019, Loss: 0.2251
116
Epoch: 000, Loss: 0.2357
Epoch: 001, Loss: 0.2385
Epoch: 002, Loss: 0.2227
Epoch: 003, Loss: 0.2427
Epoch: 004, Loss: 0.2360
Epoch: 005, Loss: 0.2258
Epoch: 006, Loss: 0.2307
Epoch: 007, Loss: 0.2368
Epoch: 008, Loss: 0.2401
Epoch: 009, Loss: 0.2267
Epoch: 010, Loss: 0.2384
Epoch: 011, Loss: 0.2175
Epoch: 012, Loss: 0.2462
Epoch: 013, Loss: 0.2396
Epoch: 014, Loss: 0.2164
Epoch: 015, Loss: 0.2328
Epoch: 016, Loss: 0.2299
Epoch: 017, Loss: 0.2419
Epoch: 018, Loss: 0.2272
Epoch: 019, Loss: 0.2280
117
Epoch: 000, Loss: 0.2419
Epoch: 001, Loss: 0.2341
Epoch: 002, Loss: 0.2470
Epoch: 003, Loss: 0.2183
Epoch: 004, Loss: 0.2307
Epoch: 005, Loss: 0.2458
Epoch: 006, Loss: 0.2447
Epoch: 007, Loss: 0.2492
Epoch: 008, Loss: 0.2416
Epoch: 009, Loss: 0.2505
Epoch: 010, Loss: 0.2424
Epoch: 011, Loss: 0.2419
Epoch: 012, Loss: 0.2382
Epoch: 013, Loss: 0.2532
Epoch: 014, Loss: 0.2606
Epoch: 015, Loss: 0.2247
Epoch: 016, Loss: 0.2389
Epoch: 017, Loss: 0.2385
Epoch: 018, Loss:

Epoch: 006, Loss: 0.2200
Epoch: 007, Loss: 0.2202
Epoch: 008, Loss: 0.2270
Epoch: 009, Loss: 0.2220
Epoch: 010, Loss: 0.2117
Epoch: 011, Loss: 0.2160
Epoch: 012, Loss: 0.2130
Epoch: 013, Loss: 0.2263
Epoch: 014, Loss: 0.2064
Epoch: 015, Loss: 0.2225
Epoch: 016, Loss: 0.2124
Epoch: 017, Loss: 0.2121
Epoch: 018, Loss: 0.2224
Epoch: 019, Loss: 0.2122
133
Epoch: 000, Loss: 0.2123
Epoch: 001, Loss: 0.1985
Epoch: 002, Loss: 0.2010
Epoch: 003, Loss: 0.2050
Epoch: 004, Loss: 0.2065
Epoch: 005, Loss: 0.2009
Epoch: 006, Loss: 0.2045
Epoch: 007, Loss: 0.2186
Epoch: 008, Loss: 0.2193
Epoch: 009, Loss: 0.2067
Epoch: 010, Loss: 0.2208
Epoch: 011, Loss: 0.1996
Epoch: 012, Loss: 0.2022
Epoch: 013, Loss: 0.2117
Epoch: 014, Loss: 0.2086
Epoch: 015, Loss: 0.2002
Epoch: 016, Loss: 0.2092
Epoch: 017, Loss: 0.2057
Epoch: 018, Loss: 0.2175
Epoch: 019, Loss: 0.2069
134
Epoch: 000, Loss: 0.2074
Epoch: 001, Loss: 0.2136
Epoch: 002, Loss: 0.2158
Epoch: 003, Loss: 0.2096
Epoch: 004, Loss: 0.2169
Epoch: 005, Loss:

Epoch: 016, Loss: 0.2202
Epoch: 017, Loss: 0.2084
Epoch: 018, Loss: 0.2240
Epoch: 019, Loss: 0.2116
149
Epoch: 000, Loss: 0.2151
Epoch: 001, Loss: 0.2089
Epoch: 002, Loss: 0.2230
Epoch: 003, Loss: 0.2048
Epoch: 004, Loss: 0.2338
Epoch: 005, Loss: 0.2154
Epoch: 006, Loss: 0.2228
Epoch: 007, Loss: 0.2261
Epoch: 008, Loss: 0.2171
Epoch: 009, Loss: 0.2138
Epoch: 010, Loss: 0.2174
Epoch: 011, Loss: 0.2159
Epoch: 012, Loss: 0.2251
Epoch: 013, Loss: 0.2231
Epoch: 014, Loss: 0.2244
Epoch: 015, Loss: 0.2198
Epoch: 016, Loss: 0.2215
Epoch: 017, Loss: 0.2221
Epoch: 018, Loss: 0.2258
Epoch: 019, Loss: 0.2384
150
Epoch: 000, Loss: 0.2524
Epoch: 001, Loss: 0.2473
Epoch: 002, Loss: 0.2492
Epoch: 003, Loss: 0.2542
Epoch: 004, Loss: 0.2517
Epoch: 005, Loss: 0.2625
Epoch: 006, Loss: 0.2459
Epoch: 007, Loss: 0.2563
Epoch: 008, Loss: 0.2499
Epoch: 009, Loss: 0.2341
Epoch: 010, Loss: 0.2481
Epoch: 011, Loss: 0.2504
Epoch: 012, Loss: 0.2561
Epoch: 013, Loss: 0.2509
Epoch: 014, Loss: 0.2398
Epoch: 015, Loss:

Epoch: 005, Loss: 0.2265
Epoch: 006, Loss: 0.2255
Epoch: 007, Loss: 0.2218
Epoch: 008, Loss: 0.2273
Epoch: 009, Loss: 0.2244
Epoch: 010, Loss: 0.1984
Epoch: 011, Loss: 0.2281
Epoch: 012, Loss: 0.2228
Epoch: 013, Loss: 0.2292
Epoch: 014, Loss: 0.2303
Epoch: 015, Loss: 0.2377
Epoch: 016, Loss: 0.2071
Epoch: 017, Loss: 0.2203
Epoch: 018, Loss: 0.2137
Epoch: 019, Loss: 0.2137
166
Epoch: 000, Loss: 0.2831
Epoch: 001, Loss: 0.2715
Epoch: 002, Loss: 0.2638
Epoch: 003, Loss: 0.2619
Epoch: 004, Loss: 0.2562
Epoch: 005, Loss: 0.2558
Epoch: 006, Loss: 0.2446
Epoch: 007, Loss: 0.2599
Epoch: 008, Loss: 0.2622
Epoch: 009, Loss: 0.2730
Epoch: 010, Loss: 0.2592
Epoch: 011, Loss: 0.2610
Epoch: 012, Loss: 0.2575
Epoch: 013, Loss: 0.2475
Epoch: 014, Loss: 0.2545
Epoch: 015, Loss: 0.2608
Epoch: 016, Loss: 0.2570
Epoch: 017, Loss: 0.2535
Epoch: 018, Loss: 0.2675
Epoch: 019, Loss: 0.2602
167
Epoch: 000, Loss: 0.2289
Epoch: 001, Loss: 0.2185
Epoch: 002, Loss: 0.2296
Epoch: 003, Loss: 0.2319
Epoch: 004, Loss:

Epoch: 012, Loss: 0.2015
Epoch: 013, Loss: 0.1932
Epoch: 014, Loss: 0.2000
Epoch: 015, Loss: 0.2031
Epoch: 016, Loss: 0.2001
Epoch: 017, Loss: 0.2045
Epoch: 018, Loss: 0.1894
Epoch: 019, Loss: 0.2062
182
Epoch: 000, Loss: 0.2309
Epoch: 001, Loss: 0.2347
Epoch: 002, Loss: 0.2274
Epoch: 003, Loss: 0.2354
Epoch: 004, Loss: 0.2309
Epoch: 005, Loss: 0.2148
Epoch: 006, Loss: 0.2117
Epoch: 007, Loss: 0.2371
Epoch: 008, Loss: 0.2184
Epoch: 009, Loss: 0.2095
Epoch: 010, Loss: 0.2304
Epoch: 011, Loss: 0.2251
Epoch: 012, Loss: 0.2269
Epoch: 013, Loss: 0.2367
Epoch: 014, Loss: 0.2379
Epoch: 015, Loss: 0.2232
Epoch: 016, Loss: 0.2103
Epoch: 017, Loss: 0.2086
Epoch: 018, Loss: 0.2251
Epoch: 019, Loss: 0.2185
183
Epoch: 000, Loss: 0.2556
Epoch: 001, Loss: 0.2528
Epoch: 002, Loss: 0.2437
Epoch: 003, Loss: 0.2618
Epoch: 004, Loss: 0.2480
Epoch: 005, Loss: 0.2485
Epoch: 006, Loss: 0.2522
Epoch: 007, Loss: 0.2443
Epoch: 008, Loss: 0.2474
Epoch: 009, Loss: 0.2448
Epoch: 010, Loss: 0.2472
Epoch: 011, Loss:

In [122]:
model.eval()

GCN(
  (conv1): GCNConv(1, 16)
  (conv2): GCNConv(16, 6)
)

In [156]:
num_pred = 0
count_pred = 0
for tg in graphs_test:
    out = model(tg)
    pred = out.argmax(dim=1)

    if torch.all(torch.eq(data.y, pred)):
        num_pred += 1
    count_pred +=1

print("Accuracy: ", num_pred/count_pred)

Accuracy:  0.618


In [154]:
torch.all(torch.eq(torch.tensor([1., 0., 0., 0., 0., 0.]), torch.tensor([1, 0, 0, 0, 0, 0])))

tensor(True)

tensor([1, 0, 0, 0, 0, 0])